In [ ]:
###This is a binary model, where CD_0.5 is dropped and only compares 0 and 1 for a more defined distinction

from PIL import Image

import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt


#set the dimensions
#160 to fit
im_ht = 160
im_wd = 160
IMG_SIZE = (160,160)
bat_sz = 16 # a little bit smaller due to amount of data
#training set
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/zachderse//Desktop/cross_sectional_data/datasets/binary', 
    batch_size = bat_sz,
    seed = 123, 
    validation_split = .2, 
    image_size = (im_ht,im_wd),
    subset = "training")

#test set
ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/zachderse//Desktop/cross_sectional_data/datasets/binary', 
    batch_size = bat_sz,
    seed = 123, 
    validation_split = .2, 
    image_size = (im_ht,im_wd),
    subset = "validation")

In [ ]:

#check classes and batches
class_names = ds_train.class_names
print(class_names)

for image_batch, labels_batch in ds_train:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

print('Number of validation batches: %d' % tf.data.experimental.cardinality(ds_valid))
print('Number of test batches: %d' % tf.data.experimental.cardinality(ds_train))


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.cache().prefetch(buffer_size=AUTOTUNE)
ds_valid = ds_valid.cache().prefetch(buffer_size=AUTOTUNE)


data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)



IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [ ]:
image_batch, label_batch = next(iter(ds_train))
feature_batch = base_model(image_batch)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')]
)


loss0, accuracy0 = model.evaluate(ds_train)
 
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
initial_epochs = 10
history = model.fit(ds_train,
                   epochs=initial_epochs,
                    validation_data=ds_valid)

In [ ]:
#visalization purposes and extra info

plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")


base_model.summary()
model.summary()